# House Sales Streamlit Dashboard

## Project Solution Planning

### Inputs

1. Business problem:
    - House Rocket is a digital platform whose business model is the purchase and sale of real estate using technology.
    - My goal is to help the company find the best business opportunities in the real estate market. The CEO of House Rocket would like to maximize the company's revenue by finding good business opportunities.
    - Their main strategy is to buy good homes in great locations at low prices and then resell them later at higher prices. The greater the difference between buying and selling, the greater the company's profit and therefore the greater its revenue.
    - The dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

### Outputs

1. A dashboard that CEO could access remotely that contains:
    -  Which houses should the House Rocket CEO buy and at what purchase price?
    - Once the house is in the company's possession, when is the best time to sell it and what would the sale price be?

### Tasks

# Imports

In [2]:
import pandas              as pd
import numpy               as np
import seaborn             as sns
import plotly.express      as px
import matplotlib.gridspec as gridspec
import panel               as pn
import ipywidgets          as widgets

import os
import folium

from folium.plugins          import MarkerCluster
from IPython.core.display    import HTML
from IPython.display         import Image
from geopy.geocoders         import Nominatim 

## Helper Functions

In [3]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [4]:
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## Loading Data

This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

source:
https://www.kaggle.com/harlfoxem/housesalesprediction

# Data Description

## Data Dimension

## Data Types

## Check NA values

## Change Dtypes

## Descriptive Statistics

# Feature Engineering

## Feature Creation